# Colour prediction for a MTG card image

---

### Import necessary libraries

In [1]:
!sudo pip install h5py
import h5py
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy import misc
import glob
from google.colab import drive
import pickle
import pandas as pd
from sklearn import preprocessing
import collections

### Mount Google drive

In [2]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Unzip the data

In [3]:
!unzip /content/gdrive/'My Drive'/'Magic Final Project'/data/images/cropped.zip

Archive:  /content/gdrive/My Drive/Magic Final Project/data/images/cropped.zip
replace cropped/39934090-36a6-4183-9176-97ea932d2685.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
df = pd.read_csv("gdrive/My Drive/Magic Final Project/data/cards.csv")

In [0]:
df = df.sort_values('Unnamed: 0')
df = df.reset_index(drop=True)

In [0]:
df1 = pd.DataFrame(df['colors'][:23000])
mask = (df1['colors'].str.len() == 1)
df1 = df1.loc[mask]

In [0]:
sgl_clr_exists= np.array(df1.index)

In [8]:
collections.Counter(df1['colors'])

Counter({'B': 3255, 'G': 3235, 'R': 3270, 'U': 3287, 'W': 3253})

In [0]:
le = preprocessing.LabelEncoder()
le.fit(df1['colors'])
df1['color_code'] = le.transform(df1['colors'])

In [0]:
image_labels_val = list(df1['color_code'])

In [0]:
image_names = list(df.iloc[sgl_clr_exists,:]['Unnamed: 0'])
image_labels1 = list(df1['color_code'])

In [12]:
print(len(image_names), len(image_labels1))

16300 16300


In [0]:
df1 = pd.DataFrame(df['colors'][23000:])
mask = (df1['colors'].str.len() == 1)
df1 = df1.loc[mask]

In [0]:
sgl_clr_exists= np.array(df1.index)

In [15]:
collections.Counter(df1['colors'])

Counter({'B': 214, 'G': 221, 'R': 211, 'U': 215, 'W': 233})

In [0]:
le = preprocessing.LabelEncoder()
le.fit(df1['colors'])
df1['color_code'] = le.transform(df1['colors'])

In [0]:
image_names_val = list(df.iloc[sgl_clr_exists,:]['Unnamed: 0'])
image_labels_test = list(df1['color_code'])

In [18]:
print(len(image_names_val), len(image_labels_val))

1094 16300


In [0]:
data = np.zeros((16300, 452, 616, 3), dtype=np.dtype('u1'))
i = 0
for name in image_names:
    img = "cropped/"+name+".jpg"
    data[i] = np.array(Image.open(img))
    i += 1

In [0]:
test_data = np.zeros((1094, 452, 616, 3), dtype=np.dtype('u1'))
i = 0
for name in image_names_val:
    img = "cropped/"+name+".jpg"
    test_data[i] = np.array(Image.open(img))
    i += 1

In [21]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), strides=(2,2), padding='same', activation=tf.nn.relu, input_shape=(452, 616, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Conv2D(32, (3,3), strides=(2,2), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Conv2D(64, (3,3), strides=(2,2), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(5, activation=tf.nn.softmax)
])
opti = tf.train.AdamOptimizer(learning_rate=0.0005)
model.compile(optimizer=opti, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 226, 308, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 113, 154, 16)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 113, 154, 16)      64        
_________________________________________________________________
dropout (Dropout)            (None, 113, 154, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 57, 77, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 38, 32)        0         
_________________________________________

In [22]:
mc = tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
history = model.fit(data, image_labels1, validation_split=0.05, batch_size=64, epochs=30, callbacks=[mc])

Train on 15485 samples, validate on 815 samples
Epoch 1/30
15424/15485 [============================>.] - ETA: 0s - loss: 1.8598 - acc: 0.3591
Epoch 00001: val_acc improved from -inf to 0.33006, saving model to best_model.h5
15485/15485 [==============================] - 38s 2ms/sample - loss: 1.8596 - acc: 0.3591 - val_loss: 1.6114 - val_acc: 0.3301
Epoch 2/30
15424/15485 [============================>.] - ETA: 0s - loss: 1.3667 - acc: 0.4610
Epoch 00002: val_acc improved from 0.33006 to 0.39387, saving model to best_model.h5
15485/15485 [==============================] - 33s 2ms/sample - loss: 1.3665 - acc: 0.4612 - val_loss: 1.4880 - val_acc: 0.3939
Epoch 3/30
15424/15485 [============================>.] - ETA: 0s - loss: 1.2044 - acc: 0.5279
Epoch 00003: val_acc did not improve from 0.39387
15485/15485 [==============================] - 34s 2ms/sample - loss: 1.2052 - acc: 0.5273 - val_loss: 1.5291 - val_acc: 0.3926
Epoch 4/30
15424/15485 [============================>.] - ETA: 0s 

In [23]:
model = tf.keras.models.load_model('best_model.h5')
model.compile(optimizer=opti, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [25]:
model.evaluate(test_data, image_labels_test)

1094/1094 [==============================] - 2s 2ms/sample - loss: 1.7141 - acc: 0.4186


[1.7140809786820979, 0.41864717]